### Imports

In [1]:
#!wget http://efrosgans.eecs.berkeley.edu/pix2pix/datasets/night2day.tar.gz

In [2]:
#!tar xvzf night2day.tar.gz

In [3]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

from PIL import Image

import torchvision
from torchvision import transforms


In [4]:
BATCH_SIZE = 1
NUM_EPOCHS = 200

### My-Utils

In [5]:
def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
def count_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    return sum([np.prod(p.size()) for p in model_parameters])

In [8]:
def my_imshow(inp, tar):
    """Imshow for Tensor."""
    inp = transforms.ToPILImage()(inp)
    tar = transforms.ToPILImage()(tar)    
    
    f, axarr = plt.subplots(1, 2) 
    f.set_figheight(5)
    f.set_figwidth(10)
    
    axarr[0].imshow(inp)
    axarr[1].imshow(tar)

    
    plt.show()
    plt.pause(0.1)

In [9]:
def count_psnr(fake, real):
    mse_loss = F.mse_loss(fake*0.5 + 0.5, real, reduction='none')
    return torch.mean(10 * torch.log10(1 / mse_loss)).item()

In [10]:
def count_cov(x, y):
    x -= torch.mean(x, dim=1, keepdim=True)
    y -= torch.mean(y, dim=1, keepdim=True)
        
    return x * y

In [11]:
# based on wiki formula
def count_ssim(x, y):
    mu_x = torch.mean(x, dim=[2,3]).squeeze()
    mu_y = torch.mean(y, dim=[2,3]).squeeze()
    
    sig_x = torch.var(x, dim=[2,3]).squeeze()
    sig_y = torch.var(y, dim=[2,3]).squeeze()
    
    sig_xy = torch.mean(count_cov(x.reshape(3, -1), y.reshape(3, -1)), 1)
    
    c1 = 0.01 ** 2
    c2 = 0.03 ** 2
        
    ssim = ((2 * mu_x * mu_y + c1) * (2 * sig_xy + c2)) / ((mu_x**2 + mu_y**2 + c1) * (sig_x + sig_y + c2))
    
    return torch.mean(ssim)

### Data

In [12]:
set_seed(21)

In [13]:
class MyDataset(torch.utils.data.Dataset):
    """Custom dataset."""

    def __init__(self, root_dir, transform=None):
        """
        Args:
            root_dir (string): Directory with train/val/test the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.root_dir = root_dir
        self.transform = transform

        
    def __len__(self):
        return len(os.listdir(path=self.root_dir))

    
    def __getitem__(self, idx):
        
        img_name = os.path.join(self.root_dir, str(idx+1)+'.jpg')
        image = Image.open(img_name)     
            
        image = torchvision.transforms.functional.to_tensor(image)

        width = int(image.size(-1) / 2)
        image, target = image[:, :, width:], image[:, :, :width]  

        if self.transform:
            image, target = self.transform(image, target)

        sample = {'x': image, 'target': target}
        
        
        return sample

In [14]:
def my_transforms_tr(sample, target):
    sample = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(sample)
    target = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(target)

    x = random.randint(0, 286 - 256)
    y = random.randint(0, 286 - 256)
    flip = random.random() > 0.5

    sample = transforms.Resize([286, 286], Image.BICUBIC)(sample)
    target = transforms.Resize([286, 286], Image.BICUBIC)(target)
    
    #sample = transforms.RandomCrop(256)(sample)
    sample = sample[:, x:256 + x, y:256 + y]
    target = target[:, x:256 + x, y:256 + y]
    
    if flip:
        actual_flip = transforms.RandomHorizontalFlip()
        sample = actual_flip(sample)
        target = actual_flip(target)
    
    return sample, target

In [15]:
def my_transforms_val(sample, target):
    sample = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(sample)
    target = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(target)
    
    return sample, target  

In [16]:
train_set = MyDataset(root_dir='facades/facades/train/', transform=my_transforms_tr)
val_set   = MyDataset(root_dir='facades/facades/val/', transform=my_transforms_val)
test_set  = MyDataset(root_dir='facades/facades/test/', transform=my_transforms_val)

In [17]:
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, 
                          shuffle=True,
                          num_workers=2, pin_memory=True)

val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, 
                        shuffle=False,
                        num_workers=1, pin_memory=True)

test_loader = DataLoader(test_set, batch_size=1, 
                        shuffle=False,
                        num_workers=1, pin_memory=True)

In [18]:
# test loader
'''for batch in train_loader:
    sample, sample_target = batch['x'][0], batch['target'][0]
    my_imshow(sample*0.5 + 0.5, sample_target*0.5 + 0.5)
    
    print()'''

"for batch in train_loader:\n    sample, sample_target = batch['x'][0], batch['target'][0]\n    my_imshow(sample*0.5 + 0.5, sample_target*0.5 + 0.5)\n    \n    print()"

### Architecture

In [19]:
def conv_bn_act(in_size, out_size, kernel_size=4, stride=2, dilation=1, padding=None, bias=False,
                is_bn=True, p_drop=None, act=None, is_up=False):
    
    padding=int((kernel_size - 1)/2)
    
    return nn.Sequential(
        nn.Conv2d(in_size, out_size, kernel_size, stride=stride, padding=padding, bias=bias) \
            if not is_up else \
            nn.ConvTranspose2d(in_size, out_size, kernel_size, stride=stride, padding=padding, bias=bias),
        nn.BatchNorm2d(out_size, eps=1e-05, momentum=0.1, affine=True) if is_bn else nn.Identity(),
        nn.Dropout(p_drop) if p_drop is not None else nn.Identity(),
        act if act is not None else nn.Identity()
    )

In [1]:
class UNet(nn.Module):
    def __init__(self, in_size=64):
        super(UNet, self).__init__()

        enc_act, dec_act = nn.LeakyReLU(0.2), nn.ReLU()
        self.encoder = nn.ModuleList([conv_bn_act(3,         in_size, is_bn=False, act=enc_act),
                                      conv_bn_act(in_size,   in_size*2, act=enc_act),
                                      conv_bn_act(in_size*2, in_size*4, act=enc_act),
                                      conv_bn_act(in_size*4, in_size*8, act=enc_act)
                                     ] +
                                     [conv_bn_act(in_size*8, in_size*8, act=enc_act)
                                      for i in range(3)] +
                                     [conv_bn_act(in_size*8, in_size*8, is_bn=False, act=enc_act)])
        
        self.first_dec_layer = conv_bn_act(in_size*8, in_size*8, p_drop=0.5, act=dec_act, is_up=True)
        
        self.decoder = nn.ModuleList([conv_bn_act(in_size*16, in_size*8, p_drop=0.5, act=dec_act, is_up=True)
                                      for i in range(3)] +
                                     [conv_bn_act(in_size*16, in_size*4, act=dec_act, is_up=True),
                                      conv_bn_act(in_size*8,  in_size*2, act=dec_act, is_up=True),
                                      conv_bn_act(in_size*4,  in_size,   act=dec_act, is_up=True),
                                      conv_bn_act(in_size*2,  3, bias=True, is_bn=False, act=nn.Tanh(), is_up=True)])
        
    
    def forward(self, x):
        N = len(self.encoder) - 1
        
        enc_outputs = []
        for layer in self.encoder:
            enc_outputs.append(x)
            x = layer(x)

        x = self.first_dec_layer(x)
        
        for i, layer in enumerate(self.decoder):
            x_concat = torch.cat([x, enc_outputs[N-i]], dim=1)
            x = layer(x_concat)
        return x

NameError: name 'nn' is not defined

In [21]:
!pip install wandb

import wandb

!wandb login 6aa2251ef1ea5e572e6a7608c0152db29bd9a294

wandb.init(project='dl-ht3')
print(1)

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: kirili4ik (use `wandb login --relogin` to force relogin)


1


In [24]:
# real_x,y ; fake_x,y . Making x->y model
class Pix2Pix(nn.Module):
    def __init__(self, in_size=64):
        super(Pix2Pix, self).__init__()

        self.netG = UNet(in_size)
        self.netD = nn.Sequential(conv_bn_act(6, in_size, is_bn=False, bias=True, act=nn.LeakyReLU(0.2)),
                                  conv_bn_act(in_size, in_size*2, act=nn.LeakyReLU(0.2)),
                                  conv_bn_act(in_size*2, in_size*4, act=nn.LeakyReLU(0.2)),
                                  conv_bn_act(in_size*4, in_size*8, stride=1, act=nn.LeakyReLU(0.2)), 
                                  conv_bn_act(in_size*8, 1, stride=1, bias=True, is_bn=False)     # no sigmoid cause of BCEwithlogits #act=nn.Sigmoid())]
                                 )
        self.cgan_loss = nn.BCEWithLogitsLoss()
        self.l1_loss  = nn.L1Loss()



    def generate(self, real_x):
        fake_y = self.netG(real_x)   # G(x)
        return fake_y


    def my_backward(self, real_x, real_y, optimizer_D, optimizer_G, lambda_par=100):
        fake_y = self.generate(real_x)

        # enable backprop for D if disabled
        for param in self.netD.parameters():
            param.requires_grad = True
        optimizer_D.zero_grad()

        ### DICRIMINATOR BACKWARD
        # real-fake
        real_fake = torch.cat((real_x, fake_y), 1)
        prediction_D = self.netD(real_fake.detach())     # detach for G backward NOT to be used
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_D_fake = self.cgan_loss(prediction_D, torch.zeros_like(prediction_D))

        # real-real
        real_real = torch.cat((real_x, real_y), 1)
        prediction_D = self.netD(real_real)
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_D_real = self.cgan_loss(prediction_D, torch.ones_like(prediction_D))

        loss_D = (loss_D_fake + loss_D_real) / 2
        loss_D.backward()
        optimizer_D.step()


        # disable backprop for D for saving Time and Resourses
        for param in self.netD.parameters():
            param.requires_grad = False

        ### GENERATOR BACKWARD
        optimizer_G.zero_grad()

        prediction_D = self.netD(real_fake)               # don't use detach for G backward TO be used
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_cgan = self.cgan_loss(prediction_D, torch.ones_like(prediction_D))

        loss_L1 = self.l1_loss(fake_y, real_y) * lambda_par
        loss_G = loss_cgan + loss_L1
        loss_G.backward()
        optimizer_G.step()
        
        ssim = count_ssim(fake_y.detach(), real_y.detach())
        
        return loss_D.item(), loss_G.item(), ssim



    def validation(self, real_x, real_y, lambda_par):
        fake_y = self.generate(real_x)

        ### DICRIMINATOR BACKWARD
        # real-fake
        real_fake = torch.cat((real_x, fake_y), 1)
        prediction_D = self.netD(real_fake.detach())
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_D_fake = self.cgan_loss(prediction_D, torch.zeros_like(prediction_D))

        # real-real
        real_real = torch.cat((real_x, real_y), 1)
        prediction_D = self.netD(real_real)
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_D_real = self.cgan_loss(prediction_D, torch.ones_like(prediction_D))

        loss_D = (loss_D_fake + loss_D_real) / 2


        ### GENERATOR BACKWARD
        prediction_D = self.netD(real_fake)
        prediction_D = prediction_D.view(prediction_D.size(0), -1)
        loss_cgan = self.cgan_loss(prediction_D, torch.ones_like(prediction_D))
        loss_L1 = self.l1_loss(fake_y, real_y) * lambda_par
        loss_G = loss_cgan + loss_L1
        
        ssim = count_ssim(fake_y.detach(), real_y.detach())
        
        return loss_D.item(), loss_G.item(), ssim

In [25]:
model = Pix2Pix()
model = model.to(device)

In [26]:
count_parameters(model)

57182660

In [27]:
wandb.watch(model)

In [28]:
from torch.nn import init

for m in model.parameters():
    classname = m.__class__.__name__
    if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
        init.normal_(m.weight.data, 0.0, 0.02)
        if hasattr(m, 'bias') and m.bias is not None:
            init.constant_(m.bias.data, 0.0)

In [30]:
opt_D = torch.optim.Adam(model.netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_G = torch.optim.Adam(model.netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [31]:
from tqdm import tqdm

In [33]:
@torch.no_grad()
def validate(model, loader, is_saving=False):
    model.netD.eval()
    total_loss_D, total_loss_G = 0, 0
    total_ssim = 0
    for i, batch in enumerate(loader):
        real_x, real_y = batch['x'].to(device), batch['target'].to(device)

        loss_D, loss_G, ssim = model.validation(real_x, real_y)

        total_loss_D = total_loss_D + loss_D
        total_loss_G = total_loss_G + loss_G
        total_ssim   = total_ssim   + ssim
        
    if is_saving:
        wandb.log({'examples':
                  [wandb.Image(transforms.ToPILImage()(real_x[0]*0.5 + 0.5), caption='real_x'),
                  wandb.Image(transforms.ToPILImage()(model.generate(real_x)[0]*0.5 + 0.5), caption='generated'),
                  wandb.Image(transforms.ToPILImage()(real_y[0]*0.5 + 0.5), caption='real_y')
                  ]})
        
    return total_loss_D / len(loader), total_loss_G / len(loader), total_ssim / len(loader)

In [34]:
@torch.no_grad()
def test(model, loader):
    model.netD.eval()
    total_loss_D, total_loss_G = 0, 0
    for i, batch in enumerate(loader):
        real_x, real_y = batch['x'].to(device), batch['target'].to(device)

        loss_D, loss_G, ssim = model.validation(real_x, real_y)

        total_loss_D = total_loss_D + loss_D
        total_loss_G = total_loss_G + loss_G
        
        if i % 2 == 0:
            wandb.log({'examples':
                  [wandb.Image(transforms.ToPILImage()(real_x[0]*0.5 + 0.5), caption='real_x'),
                  wandb.Image(transforms.ToPILImage()(model.generate(real_x)[0]*0.5 + 0.5), caption='generated'),
                  wandb.Image(transforms.ToPILImage()(real_y[0]*0.5 + 0.5), caption='real_y')
                  ]})
        
        
    return total_loss_D / len(loader), total_loss_G / len(loader)

In [35]:
### train loop
for ep_num in tqdm(range(NUM_EPOCHS)):
    model.train()
    total_loss_D, total_loss_G = 0, 0
    total_ssim = 0
    for i, batch in enumerate(train_loader):
        real_x, real_y = batch['x'].to(device), batch['target'].to(device)

        loss_D, loss_G, ssim = model.my_backward(real_x, real_y, opt_D, opt_G)

        total_loss_D = total_loss_D + loss_D
        total_loss_G = total_loss_G + loss_G
        total_ssim = total_ssim + ssim
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 15)


    if ep_num % 20 == 0:
        val_loss_D, val_loss_G, val_ssim = validate(model, val_loader, is_saving=True)
    else:
        val_loss_D, val_loss_G, val_ssim = validate(model, val_loader)  
        
    if ep_num >= 150 and ep_num % 25 == 0:
        a, b = test(model, test_loader)        
        
    wandb.log({'train_D':total_loss_D/len(train_loader),
               'train_G_l1+cgan':total_loss_G/len(train_loader),
               'train_ssim':total_ssim/len(train_loader),
               'val_D':val_loss_D,
               'val_G_l1+cgan':val_loss_G,
               'val_ssim':val_ssim})    

100%|██████████| 250/250 [2:55:47<00:00, 42.19s/it]


In [37]:
torch.save({
    'model_state_dict': model.state_dict(),
}, 'checkpoint_end_augs_200')

In [38]:
'''----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: ./datasets/facades            	[default: None]
             dataset_mode: aligned                       
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1                             
                 gan_mode: vanilla                       
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: True                          	[default: None]
                lambda_L1: 100.0                         
                load_iter: 0                             	[default: 0]
                load_size: 286                           
                       lr: 0.0002                        
           lr_decay_iters: 50                            
                lr_policy: linear                        
         max_dataset_size: inf                           
                    model: pix2pix                       	[default: cycle_gan]
                 n_epochs: 100                           
           n_epochs_decay: 100                           
               n_layers_D: 3                             
                     name: facades_label2photo           	[default: experiment_name]
                      ndf: 64                            
                     netD: basic                         
                     netG: unet_256                      
                      ngf: 64                            
               no_dropout: False                         
                  no_flip: False                         
                  no_html: False                         
                     norm: batch                         
              num_threads: 4                             
                output_nc: 3                             
                    phase: train                         
                pool_size: 0                             
               preprocess: resize_and_crop               
               print_freq: 100                           
             save_by_iter: False                         
          save_epoch_freq: 5                             
         save_latest_freq: 5000                          
           serial_batches: False                         
                   suffix:                               
         update_html_freq: 1000                          
                  verbose: False                         
----------------- End -------------------'''

'----------------- Options ---------------\n               batch_size: 1                             \n                    beta1: 0.5                           \n          checkpoints_dir: ./checkpoints                 \n           continue_train: False                         \n                crop_size: 256                           \n                 dataroot: ./datasets/facades            \t[default: None]\n             dataset_mode: aligned                       \n                direction: AtoB                          \n              display_env: main                          \n             display_freq: 400                           \n               display_id: 1                             \n            display_ncols: 4                             \n             display_port: 8097                          \n           display_server: http://localhost              \n          display_winsize: 256                           \n                    epoch: latest                     

In [39]:
'''dataset [AlignedDataset] was created
The number of training images = 400
initialize network with normal
initialize network with normal
model [Pix2PixModel] was created
---------- Networks initialized -------------
[Network G] Total number of parameters : 54.414 M
[Network D] Total number of parameters : 2.769 M
-----------------------------------------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace=True)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace=True)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (3): UnetSkipConnectionBlock(
                  (model): Sequential(
                    (0): LeakyReLU(negative_slope=0.2, inplace=True)
                    (1): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                    (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                    (3): UnetSkipConnectionBlock(
                      (model): Sequential(
                        (0): LeakyReLU(negative_slope=0.2, inplace=True)
                        (1): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                        (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                        (3): UnetSkipConnectionBlock(
                          (model): Sequential(
                            (0): LeakyReLU(negative_slope=0.2, inplace=True)
                            (1): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                            (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                            (3): UnetSkipConnectionBlock(
                              (model): Sequential(
                                (0): LeakyReLU(negative_slope=0.2, inplace=True)
                                (1): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                                (2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                                (3): UnetSkipConnectionBlock(
                                  (model): Sequential(
                                    (0): LeakyReLU(negative_slope=0.2, inplace=True)
                                    (1): Conv2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                                    (2): ReLU(inplace=True)
                                    (3): ConvTranspose2d(512, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                                    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                                  )
                                )
                                (4): ReLU(inplace=True)
                                (5): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                                (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                                (7): Dropout(p=0.5, inplace=False)
                              )
                            )
                            (4): ReLU(inplace=True)
                            (5):.log({'e2s-train_1step':loss.item()})
        #torch ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                            (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                            (7): Dropout(p=0.5, inplace=False)
                          )
                        )
                        (4): ReLU(inplace=True)
                        (5): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                        (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                        (7): Dropout(p=0.5, inplace=False)
                      )
                    )
                    (4): ReLU(inplace=True)
                    (5): ConvTranspose2d(1024, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  )
                )
                (4): ReLU(inplace=True)
                (5): ConvTranspose2d(512, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
            )
            (4): ReLU(inplace=True)
            (5): ConvTranspose2d(256, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
        )
        (2): ReLU(inplace=True)
        (3): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (4): Tanh()
      )
    )
  )
)
DataParallel(
  (module): NLayerDiscriminator(
    (model): Sequential(
      (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (1): LeakyReLU(negative_slope=0.2, inplace=True)
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (4): LeakyReLU(negative_slope=0.2, inplace=True)
      (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (7): LeakyReLU(negative_slope=0.2, inplace=True)
      (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1), bias=False)
      (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): LeakyReLU(negative_slope=0.2, inplace=True)
      (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    )
  )
)'''

"dataset [AlignedDataset] was created\nThe number of training images = 400\ninitialize network with normal\ninitialize network with normal\nmodel [Pix2PixModel] was created\n---------- Networks initialized -------------\n[Network G] Total number of parameters : 54.414 M\n[Network D] Total number of parameters : 2.769 M\n-----------------------------------------------\nDataParallel(\n  (module): UnetGenerator(\n    (model): UnetSkipConnectionBlock(\n      (model): Sequential(\n        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)\n        (1): UnetSkipConnectionBlock(\n          (model): Sequential(\n            (0): LeakyReLU(negative_slope=0.2, inplace=True)\n            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)\n            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)\n            (3): UnetSkipConnectionBlock(\n              (model): Sequential(\n                (0):

In [40]:
### НА ИНФЕРЕНСЕ ГЕНЕРАТОР В TRAIN МОДЕ, А ДИСК В EVAL